# Tutorial 4: Math Tool Agent

Large Language Models (LLMs) can serve as orchestrators, often referred to as
agents. In this tutorial, we will guide you step-by-step on how to blend
reasoning with the practical use of tools using GenC.

The `MathToolAgent` integrates reasoning with math tool. We will employ the
following components:

*   `ReAct` and `RepeatedConditionalChain` for facilitating the reasoning loop,
    enabling dynamic decision-making.
*   `RestCall`, accompanied by `parsers`, to illustrate making model calls and
    handling json request and response.
*   `LocalCache` is used to as a memory to recorde model interaction history.
*   `WolframAlpha` is utilized as a tool for answering mathematical queries.

This example is meant to demonstrate the essential ingredients to build and
experiment with agents. For production, a more robust prompt engineering is
necessary.

## Initial Setup

Before proceeding, please follow the instructions in
[Tutorial 1](https://github.com/google/genc/tree/master/genc/docs/tutorials/tutorial_1_simple_cascade.ipynb)
to set up your environment, connect Jupyter, and run the command below to run
the GenC imports we're going to use.

In [ ]:
import genc.python as genc
from genc.python import authoring
from genc.python import interop
from genc.python import runtime
from genc.python.examples import executor

## Define constants

First, let's define a few important strings - an API key for model access (like
you did in the preceding tutorials), app id for Wolfram Alpha, and the text of
the reasoning template to use.

In [ ]:
import textwrap

In [ ]:
gemini_api_key = ""
wolfram_app_id = ""

reasoning_template = """
  Solve a question answering task with interleaving Thought, Action, Observation steps.
  Thought can reason about the current situation
  Action can be only two types:
  (1) Math[query], Useful for when you need to solve math problems.
  (2) Finish[answer], which returns the answer as a number terminates.
  Here's an example:
  Question: what is the result of power of 2 + 1?
  Thought: power of 2
  Action: Math[pow(2)]
  Observation: 4
  Thought: I can now process the answer.
  Action: Math[4+1]
  Observation: 5
  Thought: Seems like I got the answer.
  Action: Finish[5]
  Please do it step by step.
  Question: {question}
  """

## Define the Agent

Now, we're going to use GenC building blocks to create an agent.

Here's the agent behavior we're aiming for:

*   We template-instruct the model to break down a question into steps.

*   In each step, the model explains its thought, and formulates a math
    equation as action.

*   The equation is then calculated by Wolfram (an external service
    you may remember from earlier tutorials).

*   These interaction history are then stored in a memory/context.

*   Wash, rinse, repease the process for another step...

In [ ]:
# Logger prints to terminal
log_it = genc.authoring.create_logger()

# Context keeps the interaction history with model.
# These Context is backed by a thread safe key value store.
read_context = genc.authoring.create_custom_function("/local_cache/read")
add_to_context = genc.authoring.create_custom_function("/local_cache/write")
evict_context = genc.authoring.create_custom_function("/local_cache/remove")

model_config = genc.authoring.create_rest_model_config(
    "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent",
    gemini_api_key,
)
model_call = genc.authoring.create_model_with_config(
    "/cloud/gemini", model_config
)

# Use WolframAlpha as a Tool to solve simple math questions.
solve_math = (
    genc.interop.langchain.CustomChain()
    | genc.authoring.create_custom_function("/react/extract_math_question")
    | genc.authoring.create_wolfram_alpha(wolfram_app_id)
    # Template Engine will extract the result from response JSON
    | genc.authoring.create_inja_template(
        "{% if queryresult.pods"
        " %}{{queryresult.pods.0.subpods.0.plaintext}}{% endif %}"
    )
)

# Define a resoning loop
# It will keep executing until a boolean op in the chain evaluates to true.
reasoning_loop = (
    genc.interop.langchain.CustomChain()
    | read_context
    | model_call
    | genc.authoring.create_custom_function("/react/parse_thought_action")
    | log_it
    | genc.authoring.create_regex_partial_match("Finish")
    | add_to_context
    | solve_math
    | genc.authoring.create_custom_function("/react/format_observation")
    | log_it
    | add_to_context
)
# If agent can't get an answer by 8th iteration, terminate.
reasoning_loop.num_iteration = 8

# Now set up the instruction tempalte and wire the agent together.
instruction_template = genc.authoring.create_prompt_template(
    reasoning_template
)

math_agent_chain = (
    genc.interop.langchain.CustomChain()
    | evict_context
    | instruction_template
    | add_to_context
    | log_it
    | reasoning_loop
)

# Compiles the agent into a portable computation, which can be run in C++/mobile
portable_ir = genc.interop.langchain.create_computation(math_agent_chain)

## Compile & Run it

The necessary components are already wired into the example runtime, in a manner
similar to what's shown in the preceding tutorial, so you can use them out of
the box (but please consult
[executor_stacks.cc](https://github.com/google/genc/tree/master/genc/cc/examples/executors/executor_stacks.cc)
if you'd like to review how this is done).

In [ ]:
# Runs the agent
runner = genc.runtime.Runner(portable_ir,
                             genc.examples.executor.create_default_executor())
runner(
    """what is the result of (square root of 4) + 3 to the power of 2 + 3 *(8 +
     4) / 2 - 7"""
)

**Sample Output**

*   Thought: I need to calculate the square root of 4 and cube 3 first.
*   Action: Math[sqrt(4)]
*   Observation: 2
*   Thought: I need to cube 3 now.
*   Action: Math[pow(3, 2)]
*   Observation: 9
*   Thought: Now I need to evaluate the expression.
*   Action: Math[(2) + (9) + (3 * (8 + 4) / 2) - 7]
*   Observation: 22
*   Thought: Seems like I got the answer.
*   Action: Finish[22]

## Save the IR to a file and deploy on Android

1. Run following code to save the IR to a file.

2. Follow instructions from
[Tutorial 1](https://github.com/google/genc/tree/master/genc/docs/tutorials/tutorial_1_simple_cascade.ipynb) to deploy the IR on an Android phone.

In [ ]:
with open("/tmp/genc_demo.pb", "wb") as f:
  f.write(portable_ir.SerializeToString())

## Run the demo on Android

Open the “GenC Demo” app (as in the first tutorial),
and type a math query.

Sample query to enter in UI:

```What is the result of (square root of 4) + 3 to the power of 2```

See the result.

## Cross-Language Interoperability

As in the earlier examples, the agent logic in the form of a portable IR can be
loaded and executed from C++ or Java, in Colab or on an Android phone (but keep
in mind that if you reconfigure it to use a smaller on-device model, more work
will be needed to engineer a working prompt).

While the above example shows authoring in Python, the same abstractions are
available in C++ (as discussed in
[api.md](https://github.com/google/genc/tree/master/genc/docs/api.md)).

## Coming next: express the agent using LangChain-based APIs

In this tutorial, we used GenC's native authoring interfaces to give you a
feel for what the native building blocks look like, and to transparently
show the exact logic of an agent.

Developer may prefer other authoring frameworks, such as LangChain, and you
may have also noticed that the tools vary from use case to use case while
the reasoning loop itself might be reusable. How can we levarage the benefit
of LangChain within GenC?

Make sure to read the next tutorial :)